In [1]:
# headers
import pandas as pd
import numpy as np
import platform as pf
import os
import xlrd
import matplotlib
import shutil
import matplotlib.dates as mdate
import matplotlib.ticker as mtick 
from matplotlib import pyplot as ppl
from sklearn import preprocessing as pprs
from scipy import stats
from datetime import datetime
from datetime import datetime

In [2]:
lineWidth = 0.5

def BarLine(title, x, bar, line, width, b_label, l_label, path2save):
    drfig = ppl.figure(figsize=(16,10))
    ax1 = drfig.add_subplot(111)
    ax1.bar(x, bar, label=b_label)
    yticks = mtick.FormatStrFormatter('%.1f%%')
    ax1.yaxis.set_major_formatter(yticks)
    ax1.legend(loc=2)
    ax1.set_ylabel(b_label)
    ax2 = ax1.twinx()
    ax2.plot(x, line, linestyle = "--", c = "red", linewidth = width, marker = '.', markersize = 3, label=l_label) 
    ax2.legend(loc=1)
    ax2.set_ylabel(l_label)
    ppl.title(title)
    ppl.xticks(rotation = -30)
    # ppl.show()
    ppl.savefig(path2save)
    ppl.close(drfig)

# parameter 'width' is useless
def Bar(title, x, bar, width, label, path2save):
    fig = ppl.figure(figsize=(16,10))
    ppl.bar(x, bar, label=label)
    ppl.legend()
    ppl.xlabel("Date")
    ppl.ylabel("Value")
    ppl.title(title)
    # ppl.xticks(rotation = -30)
    # ppl.show()
    ppl.savefig(path2save)
    ppl.close(fig)

def Line(title, x, line, width, label, path2save):
    fig = ppl.figure(figsize=(16,10))
    ppl.plot(x, line, linestyle = "--", linewidth = width, marker = '.', markersize = 3, label=label) 
    ppl.legend()
    ppl.xlabel("Date")
    ppl.ylabel("Value")
    ppl.title(title)
    # ppl.xticks(rotation = -30)
    # ppl.show()
    ppl.savefig(path2save)
    ppl.close(fig)
    
# Plot time series data (advanced features)
pd.set_option('max_row', 1000)
pd.set_option('max_column', 10)
covidDP_confirmed = '../analyzed_data/covid-19/time_series_covid19_confirmed_global.csv'
dataRootPath = '../analyzed_data/market/'
# covid_start_date = pd.read_csv(covidDP_confirmed).sort_values('Date', ascending=True).reset_index(drop=True)['Date'][0]
# Date that the data to be shown start
startDate = datetime(2019, 10, 20)
if not os.path.exists("../result/market"):
    os.makedirs("../result/market")
    print("directory 'result' created")
for root, dirs, files in os.walk(dataRootPath):
    for file in files:
        path = os.path.join(root, file)
        if path.endswith('.csv'):
            # Ignore this incomplete file
            if "WTI" in file:
                continue
            # Create a folder for plots of this indicator
            directoryPath = path.split(".c")[0]
            directoryPath = directoryPath.replace("analyzed_data", "result", 1)
            directoryPath = directoryPath.replace("\\", "/")
            if not os.path.exists(directoryPath):
                os.makedirs(directoryPath)
            name = file.split(".")[0]
            df = pd.read_csv(path)
            df["Date"] = pd.to_datetime(df["Date"])
            # re-sort
            df = df.sort_values('Date', ascending=True)
            df = df.reset_index(drop=True)
            df = df[(df['Date'] >= startDate)]
            date = df["Date"].tolist()
            
            # Close/Last in lines
            # MA5 in Lines
            # MA15 in Lines
            # MA30 in Lines
            # EWMA in Lines
            fig = ppl.figure(figsize=(16,10))
            value = df["Close/Last"].tolist()
            MA5 =  df["MA5"].tolist()
            MA15 = df["MA15"].tolist()
            MA30 = df["MA30"].tolist()
            EWMA = df["EWMA"].tolist()
            # Ugly code
            ppl.plot(date, value, label="Value", linewidth = lineWidth, linestyle = "-", marker = '.', markersize = 2)
            ppl.plot(date, MA5, label="MA5", linewidth = lineWidth, linestyle = "--", marker = ',', markersize = 2)
            ppl.plot(date, MA15, label="MA15", linewidth = lineWidth, linestyle = "-.", marker = 'o', markersize = 2)
            ppl.plot(date, MA30, label="MA30", linewidth = lineWidth, linestyle = ":", marker = 'v', markersize = 2)
            ppl.plot(date, EWMA, label="EWMA", linewidth = lineWidth, linestyle = "-", marker = '^', markersize = 2)
            ppl.xlabel("Date")
            ppl.ylabel("Value")
            ppl.legend()
            ppl.title(name + " Value & MA5, MA15, MA30, EWMA")
            # ppl.show()
            ppl.savefig(directoryPath + "/LinesCombined.png")
            ppl.close(fig)
            
            # DailyRise and DailyRiseRate
            dr = df["DailyRise"].tolist()
            drr = df["DailyRiseRate"].tolist()
            new_drr = []
            for num in drr:
                new_drr.append(num * 100)
            BarLine(name + " Daily Rise/Rise Rate", date, new_drr, dr, lineWidth, "DailyRiseRate", "DailyRise", directoryPath + "/DailyRise.png")
            
            # DailyReturn in Bars/Line
            val = df["DailyReturn"].tolist()
            Line(name + " Daily Return", date, val, lineWidth, "DailyReturn", directoryPath + "/DailyReturn.png")
            
            # DailyRippleRange in Bars/Line
            if "DailyRippleRange" in df.columns:
                val = df["DailyRippleRange"].tolist()
                Line(name + " Daily Ripple Range", date, val, lineWidth, "DailyRippleRange", directoryPath + "/DailyRippleRange.png")
            
            # DailyRippleRadio in Bars/Line
            if "DailyRippleRadio" in df.columns:
                val = df["DailyRippleRadio"].tolist()
                Line(name + " Daily Ripple Radio", date, val, lineWidth, "DailyRippleRadio", directoryPath + "/DailyRippleRadio.png")
            
            # DailyK in Lines & Bars
            val = df["DailyK"].tolist()
            Line(name + " DailyK Value", date, val, lineWidth, "DailyK", directoryPath + "/DailyK.png")
            
            # MACD in Bars
            val = df["MACD"].tolist()
            Bar(name + " MACD Value", date, val, lineWidth, "MACD", directoryPath + "/MACD.png")
            
            # K in Bars
            val = df["K"].tolist()
            Bar(name + " K Value", date, val, lineWidth, "K", directoryPath + "/K.png")
            
            # D in Bars
            val = df["D"].tolist()
            Bar(name + " D Value", date, val, lineWidth, "D", directoryPath + "/D.png")
            
            # J in Bars
            val = df["J"].tolist()
            Bar(name + " J Value", date, val, lineWidth, "J", directoryPath + "/J.png")
            
            # rs in Bars
            val = df["rs"].tolist()
            Bar(name + " rs Value", date, val, lineWidth, "rs", directoryPath + "/rs.png")
            
            # rsi in Bars
            val = df["rsi"].tolist()
            Bar(name + " rsi Value", date, val, lineWidth, "rsi", directoryPath + "/rsi.png")
            
            # MAD in Bars
            val = df["MAD"].tolist()
            Bar(name + " MAD Value", date, val, lineWidth, "MAD", directoryPath + "/MAD.png")
            
            print("Finished: " + name)
print("All done! Go check your files!")
            

directory 'result' created
Finished: CrudeOil_CL
Finished: Ethanol
Finished: Gasoline
Finished: NaturalGas
Finished: Corn
Finished: Oat
Finished: RedWheat
Finished: RoughRice
Finished: Soybeans
Finished: LiveCattle
Finished: Milk
Finished: Copper
Finished: Gold
Finished: Platinum
Finished: Silver
Finished: Bitcoin
Finished: DASH
Finished: Dogecoin
Finished: Ethereum
Finished: LiteCoin
Finished: NEO
Finished: Ripple
Finished: Tether
Finished: TRON
Finished: VeChain
Finished: AUD
Finished: BRL
Finished: CAD
Finished: CHF
Finished: EUR
Finished: GBP
Finished: INR
Finished: JPY
Finished: MXN
Finished: RUB
Finished: USL
Finished: USO
Finished: CBOE_VolatilityIndex
Finished: DowJones_IndustrialAvg
Finished: NASDAQ_100_Index
Finished: NASDAQ_100_Tech
Finished: S&P_500
All done! Go check your files!
